<a href="https://colab.research.google.com/github/sfnesbit/Wildfire-Risk-Assessment/blob/main/Google_Earth_Engine_Supplemental_Imagery_Downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
# Cloud authentication.
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
# Import, authenticate and initialize the Earth Engine library.
import ee
#Start Eath Engine w/ minimal authentication popups
try:
    ee.Initialize()
    print("Earth Engine initilized successfully!")
except ee.EEException as e:
    try:
        ee.Authenticate()
        ee.Initialize()
    except ee.EEException as e2:
        print("Earth Engine could not be initialized!")
        exit()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=Pg4xLmwlm_3PfjqYkyeltUxscM48kvWLtmHYMU76ODc&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWi-1tuzPBIWkvavRx4EBlNJEOJUK3jO611eOVnDBfwjaX5a4owmzgI

Successfully saved authorization token.


In [ ]:
import os
import cv2
import imageio
import requests

import numpy as np
from io import BytesIO
from datetime import datetime, timedelta
directory = "/content/drive/MyDrive/Thesis"

# Helper Functions

In [ ]:
# [(sw_lon, sw_lat), (ne_lon, ne_lat)]
def rectangleGeoJSON(coords):
  sw_lon = coords[0][0]
  sw_lat = coords[0][1]
  ne_lon = coords[1][0]
  ne_lat = coords[1][1]
  return ee.Geometry.Rectangle([[sw_lon, sw_lat], [ne_lon, ne_lat]])

In [ ]:
def parseFilename(filename, burned):
  tokens = filename[:-4].split("_")
  dateStr = tokens[0]
  tile = (float(tokens[1]), float(tokens[2]))
  return (tile, dateStr, burned)

# Define supplemental products to retrieve

In [ ]:
def getPrecipitationThumbnailUrl(coords, date, prevMonths=6, side=0.1):
  # Get bounding box over tile
  sw_lon = coords[0]
  sw_lat = coords[1]
  ne_lon = sw_lon + side
  ne_lat = sw_lat + side
  area = rectangleGeoJSON([(sw_lon, sw_lat),(ne_lon, ne_lat)])

  # Get start of precipitation collection range
  dateObj = datetime.strptime(date, '%Y-%m-%d')
  dateObj = dateObj + timedelta(days=(-30*prevMonths))
  startDate = datetime.strftime(dateObj, '%Y-%m-%d')

  img = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY").filterDate(startDate, date).mean()

  url = img.select('precipitation').getThumbURL({
        "region": area,
        "dimensions": "64x64",
        "format": "png",
        "min": 0, "max": 15
      })
  
  return url


def getFireHistory(coords, date, side=0.1, tolerance=0):
  # Get bounding box over tile
  sw_lon = coords[0]
  sw_lat = coords[1]
  ne_lon = sw_lon + side
  ne_lat = sw_lat + side
  area = rectangleGeoJSON([(sw_lon, sw_lat),(ne_lon, ne_lat)])

  fire = ee.ImageCollection('FIRMS').select('T21')

  # Break date into int values for easier year subtractions
  split = [int(val) for val in date.split("-")]
  year = split[0]
  month = split[1]
  day = split[2]

  # 1 year
  start = f'{year-1}-{month}-{day}'
  end = f'{year}-{((month-2)%12)+1}-{day}' # back 1 month to move far from potential current burn date
  fireOne = fire.filterDate(start, end).filterBounds(area).max().rename('1yr')

  # 5 years
  start = f'{year-5}-{month}-{day}'
  end = f'{year-1}-{month}-{day}'
  fireFive = fire.filterDate(start, end).filterBounds(area).max().rename('5yr')

  # 10 years
  start = f'{year-10}-{month}-{day}'
  end = f'{year-5}-{month}-{day}'
  fireTen = fire.filterDate(start, end).filterBounds(area).max().rename('10yr')


  # Combine images
  multiImg = fireOne.addBands(fireFive).addBands(fireTen)

  # Reduce and get info
  imgDict = multiImg.reduceRegion(
        reducer= ee.Reducer.max(),
        geometry= area,
        scale=1000
  ).getInfo()

  fireOneMax  = imgDict['1yr']
  fireFiveMax = imgDict['5yr']
  fireTenMax  = imgDict['10yr']

  # is there a burned area greater than the given tolerance?
  oneBurned  = fireOneMax  > tolerance if fireOneMax  is not None else False
  fiveBurned = fireFiveMax > tolerance if fireFiveMax is not None else False
  tenBurned  = fireTenMax  > tolerance if fireTenMax  is not None else False
  
  # Make numpy imgs with 1s if burned, 0s if not
  one  = np.reshape(np.zeros(64*64), (64,64)) + oneBurned
  five = np.reshape(np.zeros(64*64), (64,64)) + fiveBurned
  ten  = np.reshape(np.zeros(64*64), (64,64)) + tenBurned


  return np.dstack((one,five,ten))



In [ ]:
def url_to_numpy(url):
  # Download, convert to numpy
  res = requests.get(url)
  img = imageio.imread(BytesIO(res.content))

  # Grayscale there is more than 1 color band
  if len(img.shape) > 2:
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  return img

# Parse existing data

In [ ]:
burned = os.listdir(directory+'/climate/burned')
unburned = os.listdir(directory+'/climate/unburned/')

In [ ]:
burnedTiles = []
for filename in burned:
  fullPath = os.path.join(directory+'/climate/burned', filename)
  tileInfo = parseFilename(filename, True) # ((long,lat), date, burned)
  burnedTiles.append((fullPath, tileInfo))


unburnedTiles = []
for filename in unburned:
  fullPath = os.path.join(directory+'/climate/unburned', filename)
  tileInfo = parseFilename(filename, False) # ((long,lat), date, burned)
  unburnedTiles.append((fullPath, tileInfo))

In [ ]:
print(len(unburnedTiles), len(burnedTiles))

27932 3110


# Download Precipitation

## Burned

In [ ]:
burnedURLs = []

for filename, tileInfo in burnedTiles:
  burnedURLs.append((filename, getPrecipitationThumbnailUrl(tileInfo[0], tileInfo[1])))

In [ ]:
tot = len(burnedURLs)
i = 1
for filename, url in burnedURLs:
  img = url_to_numpy(url)
  oldImg = np.load(filename)
  newImg = np.dstack((oldImg, img))
  np.save(filename, newImg)
  print("\r{}/{}".format(i,tot), end='')
  i += 1

3110/3110

In [ ]:
a = np.load(burnedURLs[0][0])
a.shape

(64, 64, 13)

## Unburned

In [ ]:
unburnedURLs = []
tot = len(unburnedTiles)
i = 1
for filename, tileInfo in unburnedTiles:
  print("\r{}/{}".format(i,tot), end='')
  i += 1
  oldImg = np.load(filename)
  if oldImg.shape != (64,64,12):
    continue
  url = getPrecipitationThumbnailUrl(tileInfo[0], tileInfo[1])
  img = url_to_numpy(url)
  newImg = np.dstack((oldImg, img))
  np.save(filename, newImg)


27932/27932

In [ ]:
for filename, _ in unburnedTiles:
  if np.load(filename).shape != (64,64,13):
    print(filename)

In [ ]:
i = 1
for filename, url in unburnedURLs:
  img = url_to_numpy(url)
  oldImg = np.load(filename)
  newImg = np.dstack((oldImg, img))
  np.save(filename, newImg)
  print("\r{}/{}".format(i,tot), end='')
  i += 1

## Confirm

Checks if any files created do not have all 13 channels

In [ ]:
i = 1
tot = len(burnedTiles)
for filename, _ in burnedTiles:
  print(f'\r{i}/{tot}', end ='')
  i += 1
  npy = np.load(filename)
  if npy.shape != (64,64,13):
    print(filename)

i = 1
tot = len(unburnedTiles)
for filename, _ in unburnedTiles:
  print(f'\r{i}/{tot}', end ='')
  i += 1
  npy = np.load(filename)
  if npy.shape != (64,64,13):
    print(filename)

27932/27932

# Fire History

## Burned

In [ ]:
tot = len(burnedTiles)
i = 1
for filename, tileInfo in burnedTiles:
  print("\r{}/{}".format(i,tot), end='')
  i += 1
  oldImg = np.load(filename)
  if oldImg.shape != (64,64,13):
    continue
  history = getFireHistory(tileInfo[0], tileInfo[1])
  newImg = np.dstack((oldImg, history))
  np.save(filename, newImg)

3110/3110

In [ ]:
tot = len(burnedTiles)
i = 1
for filename, _ in burnedTiles:
  print(f"\r{i}/{tot}", end='')
  i += 1
  img = np.load(filename)
  if img.shape != (64,64,16):
    print(filename)

## Unburned

In [ ]:
def getShape(filename):
  with open(filename, 'rb') as f:
    major, minor = np.lib.format.read_magic(f)
    shape, fortran, dtype = np.lib.format.read_array_header_1_0(f)
  return shape

In [ ]:
tot = len(unburnedTiles)
i = 1
for filename, tileInfo in unburnedTiles:
  print("\r{}/{}".format(i,tot), end='')
  i += 1
  if getShape(filename) == (64,64,16):
    continue
  oldImg = np.load(filename)
  history = getFireHistory(tileInfo[0], tileInfo[1])
  newImg = np.dstack((oldImg, history))
  np.save(filename, newImg)

27932/27932

In [ ]:
print("yay")

yay
